In [1]:
# Import libraries.
import pandas as pd
import numpy as np
import scipy
import scipy.io.wavfile
import matplotlib.pyplot as plt
import math
import os
import datetime

In [25]:
# Noise Levels Messages.
def noise_levels(db):
    if(db <= 10): return "Casi tranquilo, respirando"
    elif(db > 10 and db <= 20): return "Hojas que aherrumbran, reloj que hace tictac"
    elif(db > 20 and db <= 30): return "Susurro, zona rural tranquila"
    elif(db > 30 and db <= 40): return "Biblioteca silenciosa, llamadas de aves"
    elif(db > 40 and db <= 50): return "Oficina tranquila, lluvias moderadas"
    elif(db > 50 and db <= 60): return "Conversacion normal"
    elif(db > 60 and db <= 70): return "Trafico Ocupado, aspirador"
    elif(db > 70 and db <= 80): return "Musica de la carga, despertadores"
    elif(db > 80 and db <= 90): return "Carro diesel, cortacespedes"
    elif(db > 90 and db <= 100): return "Motocicleta, secador de soplo"
    elif(db > 100 and db <= 110): return "Conciertos, nino gritando"
    elif(db > 110 and db <= 120): return "Umbral de dolor, trueno"

# Get audio-gps author.
def author(id):
    if id == 1: return "Daniel"
    elif id == 2: return "Juan Esteban"
    elif id == 3: return "Carlos"
    elif id == 4: return "Tania"

# Mean function.
def getMean(lista):
    suma = 0
    for i in lista: 
        suma += i
    return suma/len(lista)

# Convert string to datetime.
def convertToDatetime(date):
    date = date.replace("T", " ").replace("Z", " ")[:19]
    date = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    return date

def addSecondsColumn(df):

    df2 = pd.DataFrame()
    for i in range(0, len(df) - 1):
        d1 = convertToDatetime(df["time"].iloc[i])
        d2 = convertToDatetime(df["time"].iloc[i+1])
        diff = abs((d2-d1).seconds)
        data = df.iloc[i]
      
        if diff <= 1000:
            new_row = [d1 - datetime.timedelta(hours = 5), data["lat"], data["lon"], diff]
            df2 = df2.append([new_row], ignore_index=True)

    df2.columns=["time", "lat", "lon", "seconds"]
    return df2


def getNoiseDataframe(df, time_array, input_signal, compensation):

    # Initialize noise dataframe.
    noise = pd.DataFrame()

    # Seconds counter.
    seconds = 0
    
    print("Procesando... puede tomar un tiempo.")
    
    # Compute max amplitude.
    if(max(input_signal) > abs(min(input_signal))): 
        max_amplitude = max(input_signal)
    else: 
        max_amplitude = abs(min(input_signal))
        
    #input_signal_normalize = abs(input_signal / max_amplitude)
    
    # Process noise levels associate with time.
    for i in range(0, len(df)): #len(df)

        # Get audio partition.
        audio_partition = input_signal[seconds]
        audio_partition /= max_amplitude
        #print("Audion partition: ", audio_partition)

        # Compute dB of audio partition.
        INCREASE = 40
        decibeles = abs(20 * math.log(abs(i) + 1, 10)) + INCREASE + compensation
        #print(decibeles)

        # Append analysis 
        noise = noise.append([[decibeles/100, noise_levels(decibeles)]], ignore_index = True)

        # Increase seconds.
        seconds += df.iloc[i][3]

    noise.columns=["Noise", "Description"]
    return noise

def toArray(matrix):
    new_signal = list()
    for i in range(0, len(matrix)):
        new_signal.append(matrix[i][0]) #+ matrix[i][1])
        print("Progress: ", i/len(matrix)*100)
    return new_signal

In [26]:
# Load audio & GPS files.
pathFile =  "./Almacenamiento de Audios y GPS/"
csv_files = list()
sound_files = list()

# Get CSV files.
with os.scandir(pathFile) as entries:
    for entry in entries:
        if(entry.name.find("csv") > 0):
            csv_files.append(entry.name)
            sound_files.append(entry.name[0:3])

In [27]:
csv_files

['1-1-0910-0920-0.csv',
 '1-2-0910-0947-0.csv',
 '1-3-0910-1613-0.csv',
 '1-4-1017-1500-0.csv',
 '1-5-1018-1508-0.csv',
 '3-4-1010-1100-0.csv',
 '3-5-1510-1054-0.csv',
 '3-6-1710-1057-0.csv',
 '3-7-2110-0938-0.csv',
 '3-8-2210-1350-0.csv',
 '3-9-2310-1000-0.csv',
 '3-a-2410-1144-0.csv',
 '3-b-2410-1259-0.csv',
 '4-1-1110-0939-0.csv',
 '4-2-1110-1139-0.csv']

In [28]:
for i in range(0, len(csv_files)):
        
    # Print author and info.
    print("Autor: ", author(int(csv_files[i][0])))
    print(csv_files[i])
    print(sound_files[i])
    
    # Set gps and sound path file.
    gpsPath = pathFile + csv_files[i] 
    soundPath = pathFile + sound_files[i] + ".wav"
    
    # Get date, hour & compensation.
    date = csv_files[i][4:8]
    hour = csv_files[i][9:13]
    compensation = int(csv_files[i][14])
    print("Date: ", date, ", Hour: ", hour, ", Compensation: ", compensation)

    try:
        df = pd.read_csv(gpsPath, delimiter=";", error_bad_lines=False)
        df = addSecondsColumn(df)
    except:
        df = pd.read_csv(gpsPath, delimiter=",", error_bad_lines=False)
        df = addSecondsColumn(df)
    #print(df.head(10))
    
    # Load audio file.
    (sample_rate, input_signal) = scipy.io.wavfile.read(pathFile + sound_files[i] + ".wav", mmap = True)
    time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)   
    audio_duration_minutes = time_array[-1] / 60
    print("Sound ready! Duration in minutes: ", audio_duration_minutes)
    
    # Converting input signal if neccesary.
    if(author(int(csv_files[i][0])) == "Carlos"):
        input_signal = input_signal[:,0]
    
    # Append noise dataframe to dataframe.
    noise = getNoiseDataframe(df, time_array, input_signal, compensation)
    print("Noise computation ready!")
    
    df = df.join(noise)
    print("CSV Saved!")
    df.to_csv("./Resultados/" + csv_files[i].replace(".csv", "") + "-results.csv")
    
    print("------")

Autor:  Daniel
1-1-0910-0920-0.csv
1-1
Date:  0910 , Hour:  0920 , Compensation:  0
Sound ready! Duration in minutes:  15.171449357520787
Procesando... puede tomar un tiempo.
Noise computation ready!
CSV Saved!
------
Autor:  Daniel
1-2-0910-0947-0.csv
1-2
Date:  0910 , Hour:  0947 , Compensation:  0
Sound ready! Duration in minutes:  18.300862811791383
Procesando... puede tomar un tiempo.
Noise computation ready!
CSV Saved!
------
Autor:  Daniel
1-3-0910-1613-0.csv
1-3
Date:  0910 , Hour:  1613 , Compensation:  0
Sound ready! Duration in minutes:  6.514960317460318
Procesando... puede tomar un tiempo.
Noise computation ready!
CSV Saved!
------
Autor:  Daniel
1-4-1017-1500-0.csv
1-4
Date:  1017 , Hour:  1500 , Compensation:  0
Sound ready! Duration in minutes:  15.163119803476947
Procesando... puede tomar un tiempo.
Noise computation ready!
CSV Saved!
------
Autor:  Daniel
1-5-1018-1508-0.csv
1-5
Date:  1018 , Hour:  1508 , Compensation:  0
Sound ready! Duration in minutes:  4.50670483